In [1]:
import ffm
from sklearn.metrics import roc_auc_score
import gc
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


In [2]:
data_dir="/home/james/data/Malware"
train_csv=data_dir + "/train.csv"

test_csv=data_dir + "/test.csv"
label_name="HasDetections"

In [3]:
select_cols= [
    'EngineVersion',
    'HasDetections',
    'MachineIdentifier',
    'AppVersion',
    'AvSigVersion',
    'RtpStateBitfield',
    'IsSxsPassiveMode',
    'AVProductsInstalled',
    'AVProductsEnabled',
    'IsProtected',
    'SmartScreen',
    'Census_OSArchitecture',
    'Census_OSInstallTypeName',
    'Census_ActivationChannel',
    'Census_IsTouchEnabled',
    'Census_IsAlwaysOnAlwaysConnectedCapable',
    'Wdft_IsGamer']
df = pd.read_csv(train_csv,usecols=select_cols)

In [4]:
class FFMFormatPandas:
    def __init__(self):
        self.field_index_ = None
        self.feature_index_ = None
        self.y = None

    def fit(self, df, y=None):
        self.y = y
        df_ffm = df[df.columns.difference([self.y])]
        if self.field_index_ is None:
            self.field_index_ = {col: i for i, col in enumerate(df_ffm)}

        if self.feature_index_ is not None:
            last_idx = max(list(self.feature_index_.values()))

        if self.feature_index_ is None:
            self.feature_index_ = dict()
            last_idx = 0

        for col in df.columns:
            vals = df[col].unique()
            for val in vals:
                if pd.isnull(val):
                    continue
                name = '{}_{}'.format(col, val)
                if name not in self.feature_index_:
                    self.feature_index_[name] = last_idx
                    last_idx += 1
            self.feature_index_[col] = last_idx
            last_idx += 1
        return self

    def fit_transform(self, df, y=None):
        self.fit(df, y)
        return self.transform(df)

    def transform_row_(self, row, t):
        ffm = []
        if self.y != None:
            ffm.append(str(row.loc[row.index == self.y][0]))
        if self.y is None:
            ffm.append(str(0))

        for col, val in row.loc[row.index != self.y].to_dict().items():
            col_type = t[col]
            name = '{}_{}'.format(col, val)
            if col_type.kind ==  'O':
                ffm.append('{}:{}:1'.format(self.field_index_[col], self.feature_index_[name]))
            elif col_type.kind == 'i':
                ffm.append('{}:{}:{}'.format(self.field_index_[col], self.feature_index_[col], val))
        return ' '.join(ffm)

    def transform(self, df):
        t = df.dtypes.to_dict()
        return pd.Series({idx: self.transform_row_(row, t) for idx, row in df.iterrows()})

In [5]:
labels = df['HasDetections']
IDS = df['MachineIdentifier']

# Get Columns in Variable
columns = list(df.columns.values)

In [6]:
df.fillna(0,inplace=True)

In [7]:
df = df.astype('category')

In [8]:
for column in columns:
    if column == "MachineIdentifier":
        next
    if column == "HasDetections":
        next
    df[column] = pd.factorize(df[column])[0]

In [9]:
print(df.shape)

(8921483, 17)


In [10]:
ffm_train = FFMFormatPandas()
ffm_train_data = ffm_train.fit_transform(df, y='HasDetections')